### Setup

Execute streamfiles.ipynb file and then start running the current notebook.

Click "Run All" in the the companion `inputstreamfiles.ipynb` notebook in another browser tab right before running this notebook. `inputstreamfiles.ipynb` kicks off writes to the target directory every several seconds that we will use to demonstrate Auto Loader.

In [0]:
# clean up the workspace
dbutils.fs.rm("/tmp/iot_stream/", recurse=True)
dbutils.fs.rm("/tmp/iot_stream_chkpts/", recurse=True)
spark.sql(f"DROP TABLE IF EXISTS iot_stream")
spark.sql(f"DROP TABLE IF EXISTS iot_devices")
spark.sql("CREATE TABLE iot_devices USING DELTA AS SELECT * FROM json.`/databricks-datasets/iot/` WHERE 2=1")
spark.sql("SET spark.databricks.cloudFiles.schemaInference.enabled=true")
dbutils.fs.cp("/databricks-datasets/iot-stream/data-device/part-00000.json.gz",
              "/tmp/iot_stream/part-00000.json.gz", recurse=True)

Out[15]: True

In [0]:
input_data_path = "/tmp/iot_stream/"
chkpt_path = "/tmp/iot_stream_chkpts/"

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Getting your data into Delta Lake with Auto Loader and COPY INTO
Incrementally and efficiently load new data files into Delta Lake tables as soon as they arrive in your data lake (S3/Azure Data Lake/Google Cloud Storage).

<!-- <img src="https://databricks.com/wp-content/uploads/2021/02/telco-accel-blog-2-new.png" width=800/> -->
<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-data-ingestion.png" width=1000/>

<!-- <img src="https://databricks.com/wp-content/uploads/2020/02/dl-workflow2.png" width=750/> -->

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Auto Loader

In [0]:
df = (spark.readStream.format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("cloudFiles.schemaLocation", chkpt_path)
      #.option("cloudFiles.schemaEvolutionMode",)
      .load(input_data_path))

(df.writeStream.format("delta")
 .option("checkpointLocation", chkpt_path)
   .table("iot_stream"))

Out[17]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fa34765be80>

In [0]:
display(df.selectExpr("COUNT(*) AS record_count"))

record_count 1050000

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> Auto Loader with `triggerOnce`

In [0]:
df = (spark.readStream.format("cloudFiles")
      .option("cloudFiles.format", "json")
      .option("cloudFiles.schemaLocation", chkpt_path)
      .load(input_data_path))

(df.writeStream.format("delta")
   .trigger(once=True)
   .option("checkpointLocation", chkpt_path)
   .table("iot_stream"))

Out[20]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fa347624fd0>

In [0]:
%sql SELECT COUNT(*) FROM iot_stream

count(1) 650000

### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> SQL `COPY INTO` command
Retriable, idempotent, simple.

In [0]:
%sql
COPY INTO iot_devices
FROM "/databricks-datasets/iot/"
FILEFORMAT = JSON

num_affected_rows num_inserted_rows num_skipped_corrupt_files 198164 198164 0

In [0]:
%sql SELECT COUNT(*) FROM iot_devices

count(1) 198164

In [0]:
%sql DESCRIBE HISTORY iot_devices

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata engineInfo 1 2023-01-30T15:27:07.000+0000 340810560840283 kiran.kumar@insofe.edu.in COPY INTO Map() null List(4184673442652791) 0130-090555-au8in6u5 0 WriteSerializable true Map(numFiles -> 4, numOutputRows -> 198164, numOutputBytes -> 7646215, numSkippedCorruptFiles -> 0) null Databricks-Runtime/12.1.x-cpu-ml-scala2.12 0 2023-01-30T15:17:36.000+0000 340810560840283 kiran.kumar@insofe.edu.in CREATE TABLE AS SELECT Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {}) null List(4184673442652791) 0130-090555-au8in6u5 null WriteSerializable true Map(numFiles -> 0, numOutputRows -> 0, numOutputBytes -> 0) null Databricks-Runtime/12.1.x-cpu-ml-scala2.12

#### <img src="https://pages.databricks.com/rs/094-YMS-629/images/dbsquare.png" width=30/> View the documentation for [Auto Loader](https://docs.databricks.com/spark/latest/structured-streaming/auto-loader.html) and [COPY INTO](https://docs.databricks.com/spark/2.x/spark-sql/language-manual/copy-into.html).

In [0]:
# clean up workspace
dbutils.fs.rm("/tmp/iot_stream/", recurse=True)
dbutils.fs.rm("/tmp/iot_stream_chkpts/", recurse=True)
spark.sql(f"DROP TABLE IF EXISTS iot_stream")
spark.sql(f"DROP TABLE IF EXISTS iot_devices")

Out[25]: DataFrame[]